In [ ]:
!pip install --quiet --upgrade langchain-text-splitters langchain-community langgraph langchain-huggingface langchain-ollama faiss-cpu beautifulsoup4

In [ ]:
!pip install --quiet --upgrade colab-xterm
%load_ext colabxterm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.6/115.6 kB 3.7 MB/s eta 0:00:00


In [ ]:
%xterm

In [ ]:
import subprocess
import os

# Define the command to run
command = "ollama serve"

# Define the log file
log_file = "ollama.log"

#install ollama
!curl https://ollama.ai/install.sh | sh

# Start the process in the background
# We redirect stdout and stderr to a log file
with open(log_file, "w") as f:
  process = subprocess.Popen(command.split(), stdout=f, stderr=subprocess.STDOUT)

print(f"Ollama server started in the background. PID: {process.pid}")
print(f"Logs are being written to {log_file}. Use '!tail -f {log_file}' to view them.")

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 13281    0 13281    0     0  55555      0 --:--:-- --:--:-- --:--:-- 55802
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
Ollama server started in the background. PID: 2375
Logs are being written to ollama.log. Use '!tail -f ollama.log' to view them.


In [ ]:
!ollama pull smollm2
!ollama pull nomic-embed-text

In [ ]:
from langchain_ollama import ChatOllama
from langchain_ollama import OllamaEmbeddings

llm = ChatOllama(model="smollm2")
embeddings = OllamaEmbeddings(model="nomic-embed-text")

In [ ]:
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

embedding_dim = len(embeddings.embed_query("hello world"))
index = faiss.IndexFlatL2(embedding_dim)

vector_store = FAISS(
  embedding_function=embeddings,
  index=index,
  docstore=InMemoryDocstore(),
  index_to_docstore_id={},
)

# Part 1

In [ ]:
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict

loader = WebBaseLoader(
  web_path="https://lilianweng.github.io/posts/2023-06-23-agent/",
  bs_kwargs=dict(parse_only=bs4.SoupStrainer(class_=("post-content", "post-title", "post-header")))
)

docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(docs)

_ = vector_store.add_documents(documents=all_splits)

prompt = hub.pull("rlm/rag-prompt")

/usr/local/lib/python3.11/dist-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [ ]:
# Define state for application
class State(TypedDict):
  question: str
  context: List[Document]
  answer: str


# Define application steps
def retrieve(state: State):
  retrieved_docs = vector_store.similarity_search(state["question"])
  return {"context": retrieved_docs}


def generate(state: State):
  docs_content = "\n\n".join(doc.page_content for doc in state["context"])
  messages = prompt.invoke({"question": state["question"], "context": docs_content})
  response = llm.invoke(messages)
  return {"answer": response.content}


# Compile application and test
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

In [ ]:
response = graph.invoke({"question": "What is Task Decomposition?"})
print(response["context"])
print(response["answer"])

[Document(id='5fbcd3f9-2c87-4254-818b-874741dc15e3', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Component One: Planning#\nA complicated task usually involves many steps. An agent needs to know what they are and plan ahead.\nTask Decomposition#\nChain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks and shed lights into an interpretation of the model’s thinking process.\nTree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process can be BFS (breadth-first search) or DFS (depth-f

# Part 2

In [ ]:
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from typing_extensions import List, TypedDict

# Load and chunk contents of the blog
loader = WebBaseLoader(
  web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
  bs_kwargs=dict(
    parse_only=bs4.SoupStrainer(
      class_=("post-content", "post-title", "post-header")
    )
  ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(docs)

# Index chunks
_ = vector_store.add_documents(documents=all_splits)

In [ ]:
from langgraph.graph import MessagesState, StateGraph

graph_builder = StateGraph(MessagesState)

In [ ]:
from langchain_core.tools import tool

@tool(response_format="content_and_artifact")
def retrieve(query: str):
  """Retrieve information related to a query."""

  retrieved_docs = vector_store.similarity_search(query=query, k=2)

  serialized = "\n\n".join(
      (f"source: {doc.metadata}\n content: {doc.page_content}")
      for doc in retrieved_docs
  )

  return serialized, retrieved_docs

In [ ]:
from langchain_core.messages import SystemMessage
from langgraph.prebuilt import ToolNode

# Step 1: Generate an AIMessage that may include a tool-call to be sent
def query_or_respond(state: MessagesState):
  """Generate tool call for retrieval or respond."""

  llm_wtools = llm.bind_tools([retrieve])
  response = llm_wtools.invoke(state["messages"])

  # MessagesState appends messages to state instead of overwriting
  return {"messages": [response]}

# Step 2: Execute the retrieval.
tools = ToolNode([retrieve])

# Step 3: Generate a response using the retrieved content.
def generate(state: MessagesState):
  """Generate answer."""

  # Get generated ToolMessages
  recent_tool_msgs = []
  for msg in reversed(state["messages"]):
    if msg.type == "tool":
      recent_tool_msgs.append(msg)
    else:
      break

  tool_msgs = recent_tool_msgs[::-1]

  # Format into prompt
  docs_content = "\n\n".join(doc.content for doc in tool_msgs)
  sys_msg_content = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    f"{docs_content}"
  )

  conv_msgs = [
      msg
      for msg in state["messages"]
      if msg.type in ("human", "system") or (msg.type == "ai" and not msg.tool_calls)
  ]

  prompt = [SystemMessage(sys_msg_content)] + conv_msgs

  # Run
  response = llm.invoke(prompt)

  return {"messages:" [response]}